# TikTok

In [1]:
# pip install PyTikTokAPI
from TikTokAPI import TikTokAPI
import pandas as pd
import numpy as np
import math
import requests
import nest_asyncio
nest_asyncio.apply()
import pyppeteer
import pyppeteer_stealth
import time

In [2]:
data_from_excel = pd.read_excel("Chốt số liệu Thanh Toán cho INFLU.xlsx", sheet_name="View", skiprows = 2, index_col= None)

In [3]:
data = data_from_excel
data.head()

,STT,Mã INF,Kênh hợp tác,Link Video,Người phụ trách,Check,Số view (đơn vị 1000),Thành tiền,Số view (đơn vị 1000).1,Thành tiền.1,Số view (đơn vị 1000).2,Thành tiền.2,Số view (đơn vị 1000).3,Thành tiền.3,Số view (đơn vị 1000).4,Thành tiền.4,Số view (đơn vị 1000).5,Thành tiền.5
0,NaN,NaN,NaN,NaN,NaN,NaN,62166.9091,300000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,INFHUYEN641,Facebook,https://www.facebook.com/nguyen.huyenthu.716/v...,Trâm,1.0,10.0000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,INFHUYEN641,Facebook,https://www.facebook.com/100008065904636/posts...,Trâm,1.0,0.0000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,INFNHI376,TikTok,https://www.tiktok.com/@nhyyng/video/698771396...,Trâm,1.0,30.0000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69862...,Trâm,1.0,36.0000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Preprocessing data
data.dropna(subset=["Link Video"], inplace=True)
Tiktok_data = data[data['Link Video'].str.startswith('https://www.tiktok.com/')]

# get rid of unwanted new line and spacing
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace('\n', '', regex=True)
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace(' ', '', regex=True)

# get rid of '?...'
Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.split("?", expand=True)[0]

# Some links are from tiktoke main page
Tiktok_data = Tiktok_data[Tiktok_data['Link Video'].str.contains("/video/")]

<ipython-input-4-ccbfe1deb0ea>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace('\n', '', regex=True)
<ipython-input-4-ccbfe1deb0ea>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tiktok_data['Link Video'] = Tiktok_data['Link Video'].str.replace(' ', '', regex=True)
<ipython-input-4-ccbfe1deb0ea>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [5]:
Tiktok_data.head()

,STT,Mã INF,Kênh hợp tác,Link Video,Người phụ trách,Check,Số view (đơn vị 1000),Thành tiền,Số view (đơn vị 1000).1,Thành tiền.1,Số view (đơn vị 1000).2,Thành tiền.2,Số view (đơn vị 1000).3,Thành tiền.3,Số view (đơn vị 1000).4,Thành tiền.4,Số view (đơn vị 1000).5,Thành tiền.5
3,3.0,INFNHI376,TikTok,https://www.tiktok.com/@nhyyng/video/698771396...,Trâm,1.0,30.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69862...,Trâm,1.0,36.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...,Trâm,1.0,61.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6.0,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...,Trâm,1.0,71.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7.0,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69875...,Trâm,1.0,6.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
cookie = {
  "s_v_web_id": "verify_ks0hnunp_DQCutqmA_wBBb_4JlT_8dIg_ywOKAKi5n8NB",
  "tt_webid": "6989095610934822402"
}

api = TikTokAPI(cookie=cookie)

In [7]:
import datetime
begin_time = datetime.datetime.now()

dict_list = [] # Store all the info
viewcount_list = [] # Store view counts

for link in Tiktok_data['Link Video'].values:
    video_id = int(link[-19:]) # Take the Id part

    try:
        retval = api.getVideoById(video_id)
        dict_list.append(retval)
        
        view = retval['itemInfo']['itemStruct']['stats']['playCount']
        
        item = {    
                    "Link Video": link,
                    "Số View": view,
        }
        
        viewcount_list.append(item)
        
    except:
        print("Fail!" + link)
    
df = pd.DataFrame(viewcount_list)
df = Tiktok_data.merge(df, on="Link Video", how="left")
df.to_pickle("./tiktok.pkl")

print(datetime.datetime.now() - begin_time)

Fail!https://www.tiktok.com/@phun.anh/video/6987227337142193435
0:01:49.419924


In [12]:
# Function to print pretty dictionary
'''
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):
            pretty(value, indent+1)
        else:
            print('\t' * (indent+1) + str(value))
            
pretty(dict_list[0])
'''      

In [8]:
pd.read_pickle("insta.pkl").head()

,STT,Mã INF,Kênh hợp tác,Link Video,Người phụ trách,Check,Số view (đơn vị 1000),Thành tiền,Số view (đơn vị 1000).1,Thành tiền.1,Số view (đơn vị 1000).2,Thành tiền.2,Số view (đơn vị 1000).3,Thành tiền.3,Số view (đơn vị 1000).4,Thành tiền.4,Số view (đơn vị 1000).5,Thành tiền.5,Số View
0,77.0,INFTOAN565,Instagram,https://www.instagram.com/reel/CRoQb_QDvXz/,Trâm,1.0,109.0,2000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203958
1,78.0,INFTOAN565,Instagram,https://www.instagram.com/reel/CRydlrjDHH1/,Trâm,1.0,362.0,7000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1091041
2,79.0,INFTOAN565,Instagram,https://www.instagram.com/reel/CR1OPZPDuaP/,Trâm,1.0,56.0,1000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,283439
3,143.0,INFHAI912,Instagram,https://www.instagram.com/p/CRrBP9HFtOj/,Mến,1.0,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1407


# Instagram

In [9]:
# Importing packages
from selenium import webdriver
# Give access to Enter & ESC keys
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import time
import json

In [10]:
# Facebook Selenium Webscraping
insta_data = data[data['Link Video'].str.startswith('https://www.instagram.com/')]

In [11]:
# Bypass notification
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

driver = webdriver.Chrome(chrome_options=chrome_options)

# Access google chrome and open insta login
driver.get("https://www.instagram.com/accounts/login/")

def click_fb_log():
    driver.find_element_by_class_name("KPnG0").click() #using Selenium click button method
def log_in():
    driver.find_element_by_name("email").send_keys("<Type in email>")
    time.sleep(2)
    driver.find_element_by_name("pass").send_keys("<Type in pass>")
    driver.find_element_by_id('loginbutton').click()

<ipython-input-11-84a1d52b201d>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [12]:
insta_list = []

for link in insta_data["Link Video"].values:
    base_url = link
    url_json = (base_url + '?__a=1')
    driver.get(url_json)
    
    data_j = driver.find_element_by_tag_name('body').text
    data_json = json.loads(data_j)
    dict_list.append(retval)

    view = data_json["graphql"]['shortcode_media']['video_view_count']

    item = {
                "Link Video": link,
                "Số View": view,
    }
    
    insta_list.append(item)
    time.sleep(.5)
    
df = pd.DataFrame(insta_list)
df = insta_data.merge(df, on="Link Video", how="left")
df.to_pickle("insta.pkl")

In [13]:
pd.read_pickle("insta.pkl").head()

,STT,Mã INF,Kênh hợp tác,Link Video,Người phụ trách,Check,Số view (đơn vị 1000),Thành tiền,Số view (đơn vị 1000).1,Thành tiền.1,Số view (đơn vị 1000).2,Thành tiền.2,Số view (đơn vị 1000).3,Thành tiền.3,Số view (đơn vị 1000).4,Thành tiền.4,Số view (đơn vị 1000).5,Thành tiền.5,Số View
0,77.0,INFTOAN565,Instagram,https://www.instagram.com/reel/CRoQb_QDvXz/,Trâm,1.0,109.0,2000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203959
1,78.0,INFTOAN565,Instagram,https://www.instagram.com/reel/CRydlrjDHH1/,Trâm,1.0,362.0,7000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1091043
2,79.0,INFTOAN565,Instagram,https://www.instagram.com/reel/CR1OPZPDuaP/,Trâm,1.0,56.0,1000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,283441
3,143.0,INFHAI912,Instagram,https://www.instagram.com/p/CRrBP9HFtOj/,Mến,1.0,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1407


# Facebook

In [20]:
fb_data = data[data['Link Video'].str.startswith('https://www.facebook.com/')]
fb_data = fb_data[fb_data['Link Video'].str.contains("/videos/")]

In [21]:
fb_data["Link Video"].values

array(['https://www.facebook.com/nguyen.huyenthu.716/videos/2920449061508309',
       'https://www.facebook.com/100006045494111/videos/781616739195419',
       'https://www.facebook.com/tienanh.vu.334/videos/358632378979133',
       'https://www.facebook.com/moclanla/videos/810057876567671',
       'https://www.facebook.com/100034376687766/videos/1011431982958642',
       'https://www.facebook.com/100028527371304/videos/557859528559826',
       'https://www.facebook.com/Changchi.1604/videos/165010988924001',
       'https://www.facebook.com/thanh.duong.5220/videos/133547462228191',
       'https://www.facebook.com/thanhhuyen.bgvn/videos/338390987868753'],
      dtype=object)

In [22]:
driver = webdriver.Chrome(chrome_options=chrome_options)

# Access google chrome and open insta login
driver.get("https://www.facebook.com/")

<ipython-input-22-fc674300ede4>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [23]:
fb_list = []

for link in fb_data["Link Video"].values:
 
    driver.get(link)
    try:
        view_count = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[2]/div/div/div[1]/div/div/div[2]/div[1]/div/div[2]/div[2]/span/span")
        view = view_count.text[:-10].replace(",","") + "000"

        item = {    
                    "Link Video": link,
                    "Số View": view,
        }

        fb_list.append(item)
    except:
        print(link)
        
df = pd.DataFrame(fb_list)
df = fb_data.merge(df, on="Link Video", how="left")
df.to_pickle("fb.pkl")

In [24]:
tiktok = pd.read_pickle("tiktok.pkl")
insta = pd.read_pickle("insta.pkl")
fb = pd.read_pickle("fb.pkl")

In [25]:
merge1 = pd.concat([tiktok, insta, fb])

In [26]:
merge1 = merge1[["STT", "Số View"]]
merge1.dropna(subset=["Số View"], inplace=True)
merge1["Số View"] = merge1["Số View"].apply(lambda x: math.floor(int(x)/1000))
export_df = data_from_excel.merge(merge1, on="STT", how="left")

In [27]:
export_df

,STT,Mã INF,Kênh hợp tác,Link Video,Người phụ trách,Check,Số view (đơn vị 1000),Thành tiền,Số view (đơn vị 1000).1,Thành tiền.1,Số view (đơn vị 1000).2,Thành tiền.2,Số view (đơn vị 1000).3,Thành tiền.3,Số view (đơn vị 1000).4,Thành tiền.4,Số view (đơn vị 1000).5,Thành tiền.5,Số View
0,1.0,INFHUYEN641,Facebook,https://www.facebook.com/nguyen.huyenthu.716/v...,Trâm,1.0,10.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
1,2.0,INFHUYEN641,Facebook,https://www.facebook.com/100008065904636/posts...,Trâm,1.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,INFNHI376,TikTok,https://www.tiktok.com/@nhyyng/video/698771396...,Trâm,1.0,30.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0
3,4.0,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69862...,Trâm,1.0,36.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0
4,5.0,INFGIANG354,TikTok,https://www.tiktok.com/@giangdeyy0/video/69872...,Trâm,1.0,61.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,169.0,INFPHUONG332,TikTok,https://vt.tiktok.com/ZGJAJF8TU/,Mến,1.0,515.0,2500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,170.0,INFYODYVICTO,TikTok,https://www.tiktok.com/@yody_vn/video/69840261...,NaN,1.0,237.0,1000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247.0
170,171.0,INFYODYVICTO,TikTok,https://www.tiktok.com/@yody_vn/video/69842682...,NaN,1.0,180.0,500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208.0
171,172.0,INFYODYVICTO,TikTok,https://www.tiktok.com/@yody_vn/video/69847739...,NaN,1.0,1700.0,8500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900.0


In [28]:
export_df.to_excel("export_data.xlsx", index=False, sheet_name="View")